### バックテストを行う
#### https://note.com/scilabcafe/n/n41c75c5223f4

In [4]:
import pandas_datareader.data as web
import datetime

from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
#from backtesting import Backtest

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/home/bayes/.local/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/bayes/.local/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


plot backtesting graph  For just vscode enviroment only !!

In [17]:
import os
os.environ['JPY_PARENT_PID'] = '3'

import bokeh
bokeh.__version__

Loading BokehJS ...

In [2]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

#### Datas'

In [5]:
start = '2022-10-01'
end = datetime.date.today()
code = '7203.JP' # トヨタ

data = web.DataReader(code, 'stooq', start, end)

# 日付を昇順に並び替える
data.sort_index(inplace=True)

In [6]:
class SmaCross(Strategy):
    ns = 5 # 短期移動平均日数
    nl = 25 # 長期移動平均日数

    def init(self):
        # 短期移動平均
        self.smaS = self.I(SMA, self.data['Close'], self.ns)
        # 長期起動平均
        self.smaL = self.I(SMA, self.data['Close'], self.nl)

    def next(self): # チャートデータの行ごとに呼び出される
        # smaS > smaL で買う
        if crossover(self.smaS, self.smaL):
            self.buy() # 買い
        # smaS < smaL で売る
        elif crossover(self.smaL, self.smaS):
            self.position.close() # 売り

In [7]:
# バックテストを設定。バックテストクラスを初期化してインスタンスを生成
bt = Backtest(
    data, # チャートデータ
    SmaCross, # 売買戦略
    # cash=1000, # 最初の所持金
    trade_on_close=True # True：現在の終値で取引，False：次の時間の始値で取引
)

# run()メソッドでバックテストを実行
result = bt.run()

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Start                     2022-10-03 00:00:00
End                       2024-05-09 00:00:00
Duration                    584 days 00:00:00
Exposure Time [%]                   59.693878
Equity Final [$]                      11857.0
Equity Peak [$]                       13891.0
Return [%]                              18.57
Buy & Hold Return [%]               81.715169
Return (Ann.) [%]                   11.572006
Volatility (Ann.) [%]               19.762536
Sharpe Ratio                         0.585553
Sortino Ratio                        1.020833
Calmar Ratio                         0.784513
Max. Drawdown [%]                  -14.750558
Avg. Drawdown [%]                    -4.84957
Max. Drawdown Duration      201 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                      25.500911
Worst Trade [%]                     -5.163827
Avg. Trade [%]                    

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1325', ...)

In [8]:
# optimize()メソッドを実行して最適化
result = bt.optimize(
    ns=range(5, 25, 5),  # 短期移動平均日数
    nl=range(5, 75, 5),  # 長期移動平均日数
    maximize= 'Return [%]',  #※maximizeはデフォルトではSQN(System Quality Number)です
    constraint=lambda r: r.ns < r.nl  # consraintオプションを付けることで短期SMAが長期SMAより長くなることを禁止できます
)

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Backtest.optimize:   0%|          | 0/5 [00:00<?, ?it/s]

Start                     2022-10-03 00:00:00
End                       2024-05-09 00:00:00
Duration                    584 days 00:00:00
Exposure Time [%]                   58.163265
Equity Final [$]                      17529.0
Equity Peak [$]                       17529.0
Return [%]                              75.29
Buy & Hold Return [%]               81.715169
Return (Ann.) [%]                   43.450154
Volatility (Ann.) [%]               27.035924
Sharpe Ratio                         1.607127
Sortino Ratio                        4.077416
Calmar Ratio                          3.45955
Max. Drawdown [%]                  -12.559482
Avg. Drawdown [%]                    -3.24009
Max. Drawdown Duration       95 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   13
Win Rate [%]                        61.538462
Best Trade [%]                      34.155798
Worst Trade [%]                     -4.295704
Avg. Trade [%]                    

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1681', ...)

In [9]:
def RSI(close, n1, n2):
    # RSIを算出
    rsiS = ta.RSI(close, timeperiod=n1)  # 短期 n1日
    rsiL = ta.RSI(close, timeperiod=n2)  # 長期 n2日
    return rsiS, rsiL

In [10]:
class RSICross(Strategy):
    ns = 14 # 短期
    nl = 28 # 長期

    def init(self):
        # rsiS / rsiL
        self.rsiS, self.rsiL = self.I(RSI, self.data['Close'], self.ns, self.nl)


    def next(self): # チャートデータの行ごとに呼び出される
        # rsiS > rsiL で買う
        if crossover(self.rsiS, self.rsiL):
            self.buy() # 買い
        # rsiS < rsiL で売る
        elif crossover(self.rsiL, self.rsiS):
            self.position.close() # 売り

In [11]:
import talib as ta
# バックテストを設定。バックテストクラスを初期化してインスタンスを生成
bt = Backtest(
    data, # チャートデータ
    RSICross, # 売買戦略
    # cash=1000, # 最初の所持金
    trade_on_close=True # True：現在の終値で取引，False：次の時間の始値で取引
)

# run()メソッドでバックテストを実行
result = bt.run()

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Start                     2022-10-03 00:00:00
End                       2024-05-09 00:00:00
Duration                    584 days 00:00:00
Exposure Time [%]                   48.469388
Equity Final [$]                      15658.5
Equity Peak [$]                       16266.5
Return [%]                             56.585
Buy & Hold Return [%]               81.715169
Return (Ann.) [%]                   33.412502
Volatility (Ann.) [%]               21.935874
Sharpe Ratio                          1.52319
Sortino Ratio                        3.511284
Calmar Ratio                         4.104555
Max. Drawdown [%]                   -8.140347
Avg. Drawdown [%]                   -2.448186
Max. Drawdown Duration      200 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   20
Win Rate [%]                             45.0
Best Trade [%]                      37.001898
Worst Trade [%]                     -2.509148
Avg. Trade [%]                    

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2072', ...)

In [12]:
# optimize()メソッドを実行して最適化
result = bt.optimize(
    ns=range(5, 25, 5),  # 短期
    nl=range(5, 75, 5),  # 長期
    maximize= 'Return [%]',  #※maximizeはデフォルトではSQN(System Quality Number)です
    constraint=lambda r: r.ns < r.nl  # consraintオプションを付けることで短期rsiが長期rsiより長くなることを禁止できます
)

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Backtest.optimize:   0%|          | 0/5 [00:00<?, ?it/s]

Start                     2022-10-03 00:00:00
End                       2024-05-09 00:00:00
Duration                    584 days 00:00:00
Exposure Time [%]                   48.469388
Equity Final [$]                      16267.0
Equity Peak [$]                       16531.0
Return [%]                              62.67
Buy & Hold Return [%]               81.715169
Return (Ann.) [%]                   36.722665
Volatility (Ann.) [%]               22.599633
Sharpe Ratio                         1.624923
Sortino Ratio                        3.898221
Calmar Ratio                         5.055344
Max. Drawdown [%]                   -7.264127
Avg. Drawdown [%]                   -2.512789
Max. Drawdown Duration      127 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   20
Win Rate [%]                             45.0
Best Trade [%]                      38.899431
Worst Trade [%]                     -2.509148
Avg. Trade [%]                    

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2463', ...)

In [13]:
def MACD(close, n1, n2, n3):
    # MACD、シグナル、ヒストグラムを算出
    macd, macdsignal, _ = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=n3)
    return macd, macdsignal

In [14]:
class MACDCross(Strategy):
    n1 = 12 # fastperiod
    n2 = 26 # slowperiod
    n3 = 9 # signalperiod

    def init(self):
        # macd, macdsignal
        self.macd, self.macdsignal = self.I(MACD, self.data['Close'], self.n1, self.n2, self.n3)


    def next(self): # チャートデータの行ごとに呼び出される
        # macd > macdsignal で買う
        if crossover(self.macd, self.macdsignal):
            self.buy() # 買い
        # macd < macdsignal で売る
        elif crossover(self.macdsignal, self.macd):
            self.position.close() # 売り

In [15]:
# バックテストを設定。バックテストクラスを初期化してインスタンスを生成
bt = Backtest(
    data, # チャートデータ
    MACDCross, # 売買戦略
    # cash=1000, # 最初の所持金
    trade_on_close=True # True：現在の終値で取引，False：次の時間の始値で取引
)

# run()メソッドでバックテストを実行
result = bt.run()

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Start                     2022-10-03 00:00:00
End                       2024-05-09 00:00:00
Duration                    584 days 00:00:00
Exposure Time [%]                    48.72449
Equity Final [$]                      13033.5
Equity Peak [$]                       14209.5
Return [%]                             30.335
Buy & Hold Return [%]               81.715169
Return (Ann.) [%]                   18.568089
Volatility (Ann.) [%]               19.198697
Sharpe Ratio                         0.967154
Sortino Ratio                        1.863392
Calmar Ratio                         1.597027
Max. Drawdown [%]                  -11.626656
Avg. Drawdown [%]                   -2.884561
Max. Drawdown Duration      128 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   15
Win Rate [%]                        53.333333
Best Trade [%]                      15.151515
Worst Trade [%]                     -6.017562
Avg. Trade [%]                    

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2854', ...)

In [16]:
# optimize()メソッドを実行して最適化
result = bt.optimize(
    n1=range(5, 55, 5),  # fastperiod
    n2=range(10, 75, 5),  # slowperiod
    n3=range(10, 75, 5),  # signalperiod
    maximize= 'Return [%]',  #※maximizeはデフォルトではSQN(System Quality Number)です
    constraint=lambda r: r.n1 < r.n2  # consraintオプションを付けることでfastperiodがslowperiodより長くなることを禁止できます
)

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

/home/bayes/.local/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1105 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/5 [00:00<?, ?it/s]

Start                     2022-10-03 00:00:00
End                       2024-05-09 00:00:00
Duration                    584 days 00:00:00
Exposure Time [%]                   49.489796
Equity Final [$]                      16481.5
Equity Peak [$]                       17561.5
Return [%]                             64.815
Buy & Hold Return [%]               81.715169
Return (Ann.) [%]                   37.878929
Volatility (Ann.) [%]               25.675545
Sharpe Ratio                         1.475292
Sortino Ratio                        3.501668
Calmar Ratio                         2.823274
Max. Drawdown [%]                  -13.416667
Avg. Drawdown [%]                   -3.542894
Max. Drawdown Duration      141 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      42.952128
Worst Trade [%]                     20.427951
Avg. Trade [%]                    

/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/bayes/.local/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3245', ...)